In [1]:
from graph_tool.all import *
from tqdm import tqdm
import csv

In [54]:
g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
score = g.new_ep("double")
g.edge_properties["score"] = score

done_proteins_dict = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
#     counter = 0
    for row in tqdm(reader):
        if row[1] != "start":
            v1 = g.add_vertex()
            g.vp.name[v1] = row[4]
            g.vp.type[v1] = "REM"
            
            if done_proteins_dict.get(row[3]) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = row[3]
                g.vp.type[v2] = "Gene"
                done_proteins_dict[row[3]] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(row[3]))
                
            e = g.add_edge(v1, v2)
            g.ep.score[e] = row[7]

        
#         counter += 1
#         if(counter == 4):
#             break

2404862it [01:54, 20926.05it/s]


In [29]:
g.save("../data/graph-tool/allCelltypeGraph.gt")

In [4]:
g = load_graph("../data/graph-tool/allCelltypeGraph.gt")

In [28]:
g.list_properties()

name           (vertex)  (type: string)
type           (vertex)  (type: string)
score          (edge)    (type: double)


In [6]:
# for e in g.edges():
#     print(g.ep.score[e])
rowCount = sum(1 for e in g.edges())
print(rowCount)

2404861
